In [2]:
# importing packages 
import numpy as np
import pandas as pd 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules 


In [3]:
data= pd.read_excel('Online Retail.xlsx')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# Let's do some exploratory analysis and prepare the data to build a recommendation system 
# removing extra space from beginning and end in Description column 

data['Description']=data['Description'].str.strip()

In [5]:
# remving duplicate invoices 
data.dropna(axis=0,subset=['InvoiceNo'],inplace=True)

In [6]:
# Coverting invoice number to a string 
data['InvoiceNo']=data['InvoiceNo'].astype('str')

In [7]:
# Removing credit transactions 
data['InvoiceNo']=data[~data['InvoiceNo'].str.contains('C')]

In [8]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [9]:
# Let's check the number of invoices per country 
data['Country'].value_counts()

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

In [11]:
# I will restict the data to consider only data for Germany
basket= (data[data['Country']=='Germany'].groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))

In [12]:
basket.head()

Description,3 HOOK HANGER MAGIC GARDEN,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,ASSORTED COLOUR LIZARD SUCTION HOOK,CHILDREN'S CIRCUS PARADE MUG,HOMEMADE JAM SCENTED CANDLES,HOT WATER BOTTLE BABUSHKA,JUMBO BAG OWLS,JUMBO BAG WOODLAND ANIMALS,MULTI COLOUR SILVER T-LIGHT HOLDER,...,CHRISTMAS STAR WISH LIST CHALKBOARD,SET OF 2 TEA TOWELS PING MICROWAVE,SKULL DESIGN TV DINNER TRAY,ELEPHANT BIRTHDAY CARD,MINI JIGSAW DINOSAUR,SET OF 6 RIBBONS PARTY,SET OF 6 RIBBONS PERFECTLY PRETTY,TRIANGULAR POUFFE VINTAGE,HAND WARMER UNION JACK,RIBBON REEL FLORA + FAUNA
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,12.0,12.0,12.0,24.0,12.0,12.0,4.0,10.0,10.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Since we are only concerned with whether the particular item is ordered or not, we will convert all positive item quantities to 1 and keep 0 quantity as 0

# defining the function for the same
def conv_units(x):
    if x<=0:
        return 0
    if x >=1:
        return 1

In [14]:
basket_sets=basket.applymap(conv_units)


In [15]:
# Dropping POSTAGE as it is seems irrelevant 
basket_sets.drop('POSTAGE',inplace=True,axis=1)

In [16]:
# We can now train the model 

my_freq_items=apriori(basket_sets,min_support=0.07,use_colnames=True)

In [17]:
# let's generate the rules 
my_rules= association_rules(my_freq_items,metric='lift',min_threshold=1)

In [19]:
my_rules.head(100)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4 FRUITS),0.245077,0.157549,0.131291,0.535714,3.400298,0.092679,1.814509
1,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,0.092679,4.529540
2,(ROUND SNACK BOXES SET OF4 WOODLAND),(SPACEBOY LUNCH BOX),0.245077,0.102845,0.070022,0.285714,2.778116,0.044817,1.256018
3,(SPACEBOY LUNCH BOX),(ROUND SNACK BOXES SET OF4 WOODLAND),0.102845,0.245077,0.070022,0.680851,2.778116,0.044817,2.365427
4,(ROUND SNACK BOXES SET OF4 WOODLAND),(PLASTERS IN TIN WOODLAND ANIMALS),0.245077,0.137856,0.074398,0.303571,2.202098,0.040613,1.237951
5,(PLASTERS IN TIN WOODLAND ANIMALS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.137856,0.245077,0.074398,0.539683,2.202098,0.040613,1.640006


In [20]:
# Now we can make recommendations
# filtering rules based on a  condition 

my_rules[(my_rules['lift']>=3) & (my_rules['confidence']>=0.3)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4 FRUITS),0.245077,0.157549,0.131291,0.535714,3.400298,0.092679,1.814509
1,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,0.092679,4.529540
